# 🎨 Notebook 10: Segmentace Obrázků

**Autor:** Praut s.r.o. - AI Integration & Business Automation

V tomto notebooku se naučíme používat modely pro segmentaci obrázků - od sémantické segmentace až po instance segmentation. Ukážeme si praktické aplikace jako odstranění pozadí, segmentaci produktů a automatickou editaci.

## Co se naučíte:
- Sémantická segmentace s SegFormer
- Instance segmentation s Mask2Former
- Panoptická segmentace
- Odstranění pozadí (background removal)
- Praktické automatizace

In [ ]:
# Instalace potřebných knihoven
!pip install -q transformers accelerate torch torchvision pillow requests matplotlib scipy

In [ ]:
import torch
from transformers import pipeline
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy import ndimage

# Detekce zařízení
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Používám zařízení: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Pomocné funkce

In [ ]:
def load_image(source):
    """Načte obrázek z URL nebo lokální cesty."""
    if source.startswith('http'):
        response = requests.get(source, timeout=10)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(source)
    
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return image

# Barevná paleta pro segmentaci
def create_color_palette(num_classes):
    """Vytvoří barevnou paletu pro vizualizaci."""
    np.random.seed(42)
    palette = np.random.randint(0, 255, size=(num_classes, 3), dtype=np.uint8)
    palette[0] = [0, 0, 0]  # Pozadí černé
    return palette

def visualize_segmentation(image, segmentation_map, labels=None, alpha=0.5):
    """
    Vizualizuje segmentační masku přes obrázek.
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Originál
    axes[0].imshow(image)
    axes[0].set_title('Originál')
    axes[0].axis('off')
    
    # Segmentační maska
    num_classes = int(segmentation_map.max()) + 1
    palette = create_color_palette(num_classes)
    colored_mask = palette[segmentation_map]
    
    axes[1].imshow(colored_mask)
    axes[1].set_title('Segmentační maska')
    axes[1].axis('off')
    
    # Overlay
    overlay = np.array(image.resize((segmentation_map.shape[1], segmentation_map.shape[0])))
    overlay = (overlay * (1 - alpha) + colored_mask * alpha).astype(np.uint8)
    
    axes[2].imshow(overlay)
    axes[2].set_title('Překryv')
    axes[2].axis('off')
    
    # Legenda
    if labels:
        unique_classes = np.unique(segmentation_map)
        patches_list = []
        for cls in unique_classes:
            if cls < len(labels):
                color = palette[cls] / 255.0
                patches_list.append(mpatches.Patch(color=color, label=labels[cls]))
        if patches_list:
            fig.legend(handles=patches_list, loc='center right', bbox_to_anchor=(1.15, 0.5))
    
    plt.tight_layout()
    plt.show()

print("✅ Pomocné funkce připraveny")

## 2. Sémantická segmentace s SegFormer

In [ ]:
# Načtení SegFormer modelu (ADE20K dataset - 150 tříd)
segformer_processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
segformer_model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

if device == "cuda":
    segformer_model = segformer_model.to(device)

# ADE20K labels (prvních 20)
ADE20K_LABELS = [
    'wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road', 'bed',
    'windowpane', 'grass', 'cabinet', 'sidewalk', 'person', 'earth',
    'door', 'table', 'mountain', 'plant', 'curtain', 'chair', 'car',
    'water', 'painting', 'sofa', 'shelf', 'house', 'sea', 'mirror',
    'rug', 'field', 'armchair', 'seat', 'fence', 'desk', 'rock',
    'wardrobe', 'lamp', 'bathtub', 'railing', 'cushion', 'base',
    'box', 'column', 'signboard', 'chest of drawers', 'counter',
    'sand', 'sink', 'skyscraper', 'fireplace'
]

print("✅ SegFormer model načten")
print(f"   Počet tříd: {segformer_model.config.num_labels}")

In [ ]:
def segment_image_segformer(image):
    """
    Provede sémantickou segmentaci pomocí SegFormer.
    """
    if isinstance(image, str):
        image = load_image(image)
    
    # Příprava vstupu
    inputs = segformer_processor(images=image, return_tensors="pt")
    if device == "cuda":
        inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Inference
    with torch.no_grad():
        outputs = segformer_model(**inputs)
    
    # Post-processing
    logits = outputs.logits
    
    # Upsampling na původní velikost
    upsampled_logits = torch.nn.functional.interpolate(
        logits,
        size=image.size[::-1],
        mode='bilinear',
        align_corners=False
    )
    
    # Argmax pro získání tříd
    segmentation = upsampled_logits.argmax(dim=1)[0].cpu().numpy()
    
    return segmentation

# Test
test_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/300px-PNG_transparency_demonstration_1.png"
room_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/GoldenGateBridge-001.jpg/1280px-GoldenGateBridge-001.jpg"

try:
    image = load_image(room_url)
    segmentation = segment_image_segformer(image)
    
    print(f"Velikost segmentace: {segmentation.shape}")
    print(f"Unikátní třídy: {np.unique(segmentation)}")
    
    visualize_segmentation(image, segmentation, ADE20K_LABELS)
except Exception as e:
    print(f"Chyba: {e}")
    # Záložní test
    cat_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"
    image = load_image(cat_url)
    segmentation = segment_image_segformer(image)
    visualize_segmentation(image, segmentation, ADE20K_LABELS)

## 3. Pipeline API pro segmentaci

In [ ]:
# Jednodušší přístup pomocí pipeline
segmenter = pipeline(
    "image-segmentation",
    model="nvidia/segformer-b0-finetuned-ade-512-512",
    device=0 if device == "cuda" else -1
)

print("✅ Segmentation pipeline připraven")

In [ ]:
# Test pipeline
cat_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"
image = load_image(cat_url)

results = segmenter(image)

print(f"Nalezeno {len(results)} segmentů:")
for r in results[:10]:  # Prvních 10
    if r['score'] is not None:
        print(f"  - {r['label']}: {r['score']:.1%}")
    else:
        print(f"  - {r['label']}")

## 4. Odstranění pozadí (Background Removal)

In [ ]:
class BackgroundRemover:
    """
    Třída pro odstranění pozadí z obrázků.
    """
    
    def __init__(self):
        # Použijeme segmentaci pro detekci objektů
        self.segmenter = segmenter
        
        # Třídy, které chceme zachovat (popředí)
        self.foreground_classes = [
            'person', 'cat', 'dog', 'car', 'bird', 'horse', 'sheep', 'cow',
            'elephant', 'bear', 'zebra', 'giraffe', 'chair', 'couch', 'bed',
            'dining table', 'toilet', 'tv', 'laptop', 'cell phone', 'book',
            'bottle', 'cup', 'plant'
        ]
    
    def remove_background(self, image, target_class=None, background_color=(255, 255, 255)):
        """
        Odstraní pozadí z obrázku.
        
        Args:
            image: Obrázek nebo URL
            target_class: Konkrétní třída k zachování (None = všechny popředí)
            background_color: Barva nového pozadí (RGB) nebo None pro průhlednost
        """
        if isinstance(image, str):
            image = load_image(image)
        
        # Segmentace
        results = self.segmenter(image)
        
        # Vytvoření masky
        mask = np.zeros((image.size[1], image.size[0]), dtype=np.uint8)
        
        for segment in results:
            label = segment['label'].lower()
            segment_mask = np.array(segment['mask'])
            
            # Filtrování podle třídy
            if target_class:
                if label == target_class.lower():
                    mask = np.maximum(mask, segment_mask)
            else:
                # Zachovat všechny popředí třídy
                for fg_class in self.foreground_classes:
                    if fg_class in label:
                        mask = np.maximum(mask, segment_mask)
                        break
        
        # Aplikace masky
        image_array = np.array(image)
        
        if background_color is None:
            # Průhledné pozadí (RGBA)
            result = np.zeros((image.size[1], image.size[0], 4), dtype=np.uint8)
            result[:, :, :3] = image_array
            result[:, :, 3] = mask * 255
            return Image.fromarray(result, 'RGBA')
        else:
            # Barevné pozadí
            result = np.array([
                [background_color if m == 0 else tuple(p) 
                 for m, p in zip(mask_row, image_row)]
                for mask_row, image_row in zip(mask, image_array)
            ], dtype=np.uint8)
            return Image.fromarray(result)
    
    def get_mask(self, image, target_class=None):
        """
        Vrátí pouze masku bez aplikace.
        """
        if isinstance(image, str):
            image = load_image(image)
        
        results = self.segmenter(image)
        mask = np.zeros((image.size[1], image.size[0]), dtype=np.uint8)
        
        for segment in results:
            label = segment['label'].lower()
            segment_mask = np.array(segment['mask'])
            
            if target_class:
                if label == target_class.lower():
                    mask = np.maximum(mask, segment_mask)
            else:
                for fg_class in self.foreground_classes:
                    if fg_class in label:
                        mask = np.maximum(mask, segment_mask)
                        break
        
        return mask

# Test
bg_remover = BackgroundRemover()
print("✅ BackgroundRemover připraven")

In [ ]:
# Test odstranění pozadí
try:
    mask = bg_remover.get_mask(image)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image)
    axes[0].set_title('Originál')
    axes[0].axis('off')
    
    axes[1].imshow(mask, cmap='gray')
    axes[1].set_title('Maska')
    axes[1].axis('off')
    
    # Aplikace masky
    masked = np.array(image) * np.expand_dims(mask, -1)
    axes[2].imshow(masked)
    axes[2].set_title('S odstraněným pozadím')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Chyba při odstranění pozadí: {e}")

## 5. Instance Segmentation

In [ ]:
# Instance segmentation pipeline
instance_segmenter = pipeline(
    "image-segmentation",
    model="facebook/mask2former-swin-base-coco-instance",
    device=0 if device == "cuda" else -1
)

print("✅ Instance segmentation pipeline připraven")

In [ ]:
def visualize_instances(image, segments):
    """
    Vizualizuje jednotlivé instance.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    
    # Originál
    axes[0].imshow(image)
    axes[0].set_title('Originál')
    axes[0].axis('off')
    
    # Instance
    overlay = np.array(image).copy().astype(float)
    
    colors = plt.cm.tab20(np.linspace(0, 1, len(segments)))
    legend_patches = []
    
    for i, segment in enumerate(segments):
        mask = np.array(segment['mask'])
        color = colors[i][:3]
        
        # Aplikace barvy na masku
        for c in range(3):
            overlay[:, :, c] = np.where(
                mask > 0,
                overlay[:, :, c] * 0.5 + color[c] * 255 * 0.5,
                overlay[:, :, c]
            )
        
        # Legenda
        score = segment.get('score', None)
        label = segment['label']
        if score:
            legend_patches.append(mpatches.Patch(color=color, label=f"{label}: {score:.1%}"))
        else:
            legend_patches.append(mpatches.Patch(color=color, label=label))
    
    axes[1].imshow(overlay.astype(np.uint8))
    axes[1].set_title('Instance Segmentation')
    axes[1].axis('off')
    axes[1].legend(handles=legend_patches, loc='upper right', fontsize=8)
    
    plt.tight_layout()
    plt.show()

# Test
try:
    instances = instance_segmenter(image)
    print(f"Nalezeno {len(instances)} instancí")
    
    for inst in instances[:5]:
        score = inst.get('score', 'N/A')
        if isinstance(score, float):
            print(f"  - {inst['label']}: {score:.1%}")
        else:
            print(f"  - {inst['label']}")
    
    visualize_instances(image, instances[:10])
except Exception as e:
    print(f"Chyba: {e}")

## 6. Praktická automatizace: Product Image Processor

In [ ]:
class ProductImageProcessor:
    """
    Procesor produktových obrázků pro e-commerce.
    """
    
    def __init__(self):
        self.segmenter = segmenter
    
    def extract_product(self, image, padding=10):
        """
        Extrahuje produkt z obrázku.
        """
        if isinstance(image, str):
            image = load_image(image)
        
        # Segmentace
        results = self.segmenter(image)
        
        # Najdi největší segment (pravděpodobně produkt)
        largest_segment = None
        largest_area = 0
        
        for segment in results:
            mask = np.array(segment['mask'])
            area = mask.sum()
            if area > largest_area:
                largest_area = area
                largest_segment = segment
        
        if largest_segment is None:
            return None, None
        
        mask = np.array(largest_segment['mask'])
        
        # Najdi bounding box
        rows = np.any(mask, axis=1)
        cols = np.any(mask, axis=0)
        
        if not rows.any() or not cols.any():
            return None, None
        
        y1, y2 = np.where(rows)[0][[0, -1]]
        x1, x2 = np.where(cols)[0][[0, -1]]
        
        # Přidej padding
        y1 = max(0, y1 - padding)
        y2 = min(image.size[1], y2 + padding)
        x1 = max(0, x1 - padding)
        x2 = min(image.size[0], x2 + padding)
        
        # Oříznutí
        cropped = image.crop((x1, y1, x2, y2))
        cropped_mask = mask[y1:y2, x1:x2]
        
        return cropped, cropped_mask
    
    def create_product_image(self, image, background_color=(255, 255, 255), size=(800, 800)):
        """
        Vytvoří profesionální produktový obrázek.
        """
        # Extrahuj produkt
        product, mask = self.extract_product(image)
        
        if product is None:
            return None
        
        # Vytvoř nové pozadí
        result = Image.new('RGB', size, background_color)
        
        # Škálování produktu
        product_ratio = product.size[0] / product.size[1]
        target_size = int(size[0] * 0.8)  # 80% velikosti
        
        if product_ratio > 1:
            new_width = target_size
            new_height = int(target_size / product_ratio)
        else:
            new_height = target_size
            new_width = int(target_size * product_ratio)
        
        product = product.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Centrování
        x = (size[0] - new_width) // 2
        y = (size[1] - new_height) // 2
        
        result.paste(product, (x, y))
        
        return result
    
    def batch_process(self, images, output_dir=None):
        """
        Dávkové zpracování produktových obrázků.
        """
        results = []
        
        for i, img_source in enumerate(images):
            try:
                processed = self.create_product_image(img_source)
                
                if processed and output_dir:
                    output_path = f"{output_dir}/product_{i:04d}.jpg"
                    processed.save(output_path, quality=95)
                
                results.append({
                    "index": i,
                    "status": "success",
                    "image": processed
                })
            except Exception as e:
                results.append({
                    "index": i,
                    "status": "error",
                    "error": str(e)
                })
        
        return results

# Test
processor = ProductImageProcessor()
print("✅ ProductImageProcessor připraven")

In [ ]:
# Test extrakce produktu
try:
    product, mask = processor.extract_product(image)
    
    if product:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(image)
        axes[0].set_title('Originál')
        axes[0].axis('off')
        
        axes[1].imshow(product)
        axes[1].set_title('Extrahovaný produkt')
        axes[1].axis('off')
        
        # Profesionální obrázek
        pro_image = processor.create_product_image(image)
        if pro_image:
            axes[2].imshow(pro_image)
            axes[2].set_title('Profesionální produkt')
            axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
    else:
        print("Nepodařilo se extrahovat produkt")
except Exception as e:
    print(f"Chyba: {e}")

## 7. Analýza kompozice obrázku

In [ ]:
class CompositionAnalyzer:
    """
    Analyzátor kompozice obrázku pomocí segmentace.
    """
    
    def __init__(self):
        self.segmenter = segmenter
    
    def analyze(self, image):
        """
        Analyzuje kompozici obrázku.
        """
        if isinstance(image, str):
            image = load_image(image)
        
        results = self.segmenter(image)
        
        width, height = image.size
        total_pixels = width * height
        
        # Analýza segmentů
        segment_analysis = []
        
        for segment in results:
            mask = np.array(segment['mask'])
            area = mask.sum()
            coverage = area / total_pixels * 100
            
            # Pozice centroidu
            if area > 0:
                y_coords, x_coords = np.where(mask > 0)
                centroid_x = x_coords.mean() / width
                centroid_y = y_coords.mean() / height
            else:
                centroid_x, centroid_y = 0.5, 0.5
            
            segment_analysis.append({
                "label": segment['label'],
                "coverage_percent": coverage,
                "centroid": (centroid_x, centroid_y),
                "position": self._get_position(centroid_x, centroid_y)
            })
        
        # Seřazení podle pokrytí
        segment_analysis.sort(key=lambda x: x['coverage_percent'], reverse=True)
        
        # Celková analýza
        main_subjects = [s for s in segment_analysis if s['coverage_percent'] > 5]
        
        return {
            "total_segments": len(results),
            "main_subjects": main_subjects,
            "dominant_element": segment_analysis[0] if segment_analysis else None,
            "composition_type": self._determine_composition(segment_analysis),
            "all_segments": segment_analysis
        }
    
    def _get_position(self, x, y):
        """Určí pozici v obrázku."""
        h_pos = "left" if x < 0.33 else ("right" if x > 0.66 else "center")
        v_pos = "top" if y < 0.33 else ("bottom" if y > 0.66 else "middle")
        return f"{v_pos}-{h_pos}"
    
    def _determine_composition(self, segments):
        """Určí typ kompozice."""
        if not segments:
            return "empty"
        
        main = segments[0]
        
        if main['coverage_percent'] > 50:
            return "single-subject-dominant"
        elif main['coverage_percent'] > 20:
            return "single-subject"
        elif len([s for s in segments if s['coverage_percent'] > 10]) > 2:
            return "multi-subject"
        else:
            return "scattered"

# Test
comp_analyzer = CompositionAnalyzer()

try:
    analysis = comp_analyzer.analyze(image)
    
    print("=" * 50)
    print("ANALÝZA KOMPOZICE")
    print("=" * 50)
    print(f"Typ kompozice: {analysis['composition_type']}")
    print(f"Počet segmentů: {analysis['total_segments']}")
    
    if analysis['dominant_element']:
        dom = analysis['dominant_element']
        print(f"\nDominantní element: {dom['label']}")
        print(f"  Pokrytí: {dom['coverage_percent']:.1f}%")
        print(f"  Pozice: {dom['position']}")
    
    if analysis['main_subjects']:
        print("\nHlavní subjekty:")
        for subj in analysis['main_subjects'][:5]:
            print(f"  - {subj['label']}: {subj['coverage_percent']:.1f}% ({subj['position']})")
except Exception as e:
    print(f"Chyba: {e}")

## 8. Depth Estimation (bonus)

In [ ]:
# Depth estimation pipeline
try:
    depth_estimator = pipeline(
        "depth-estimation",
        model="Intel/dpt-hybrid-midas",
        device=0 if device == "cuda" else -1
    )
    print("✅ Depth estimation pipeline připraven")
except Exception as e:
    print(f"Depth estimation není k dispozici: {e}")
    depth_estimator = None

In [ ]:
if depth_estimator:
    try:
        depth_result = depth_estimator(image)
        depth_map = np.array(depth_result['depth'])
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        axes[0].imshow(image)
        axes[0].set_title('Originál')
        axes[0].axis('off')
        
        im = axes[1].imshow(depth_map, cmap='plasma')
        axes[1].set_title('Hloubková mapa')
        axes[1].axis('off')
        plt.colorbar(im, ax=axes[1], label='Vzdálenost')
        
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Chyba při odhadu hloubky: {e}")
else:
    print("Depth estimation přeskočen")

## 9. Utility funkce pro export

In [ ]:
def save_segmentation_results(image, segments, output_dir, base_name="segmentation"):
    """
    Uloží výsledky segmentace.
    """
    import os
    import json
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Uložení originálního obrázku
    if isinstance(image, str):
        image = load_image(image)
    image.save(f"{output_dir}/{base_name}_original.jpg")
    
    # Uložení jednotlivých masek
    metadata = []
    
    for i, segment in enumerate(segments):
        mask = segment['mask']
        label = segment['label']
        
        # Uložení masky
        mask_img = Image.fromarray((np.array(mask) * 255).astype(np.uint8))
        mask_path = f"{output_dir}/{base_name}_mask_{i:03d}_{label}.png"
        mask_img.save(mask_path)
        
        metadata.append({
            "index": i,
            "label": label,
            "score": segment.get('score'),
            "mask_file": os.path.basename(mask_path)
        })
    
    # Uložení metadat
    with open(f"{output_dir}/{base_name}_metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"Uloženo {len(segments)} segmentů do {output_dir}")
    return metadata

print("✅ Export funkce připraveny")

## Shrnutí

V tomto notebooku jsme se naučili:

1. **Sémantická segmentace** - klasifikace každého pixelu do třídy
2. **SegFormer model** - efektivní transformer pro segmentaci
3. **Instance segmentation** - rozlišení jednotlivých objektů
4. **Odstranění pozadí** - automatické odstranění pozadí z produktových fotek
5. **Analýza kompozice** - vyhodnocení rozložení prvků v obrázku
6. **Depth estimation** - odhad hloubky scény
7. **Praktické automatizace** - zpracování produktových obrázků pro e-shop

### Další kroky
- Notebook 11: Text Embeddings a Sémantické Vyhledávání